In [37]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

In [7]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True  # applies the standard scalar to data if true
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 58187


In [9]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [15]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)


[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [19]:
# scale training data
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler() # z = (x - u) / s
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

True
True


In [21]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test  data shape: {item_test.shape}")

movie/item training data shape: (46549, 17)
movie/item test  data shape: (11638, 17)


In [27]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(46549, 1) (11638, 1)


In [51]:
# Define the custom L2 normalization layer
class L2Normalize(layers.Layer):
    def __init__(self, axis=1):
        super(L2Normalize, self).__init__()
        self.axis = axis

    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=self.axis)

num_user_features = 14
num_item_features = 16
num_outputs = 32

tf.random.set_seed(1)

# Define the user and item networks
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

item_NN = tf.keras.models.Sequential([  
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# Create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = L2Normalize(axis=1)(vu)  # Apply L2 normalization here

# Create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = L2Normalize(axis=1)(vm)  # Apply L2 normalization here

# Compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Specify the inputs and output of the model
model = Model([input_user, input_item], output)

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)   │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_15 (InputLayer)   │ (None, 16)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_12 (Sequential)    │ (None, 32)                │          40,864 │ input_layer_13[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_13 (Sequential)    │ (None, 32)                │          41,376 │ input_layer_15[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ l2_normalize_4 (L2Normalize)  │ (None, 32)                │               0 │ sequential_12[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ l2_normalize_5 (L2Normalize)  │ (None, 32)                │               0 │ sequential_13[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot_2 (Dot)                   │ (None, 1)                 │               0 │ l2_normalize_4[0][0],      │
│                               │                           │                 │ l2_normalize_5[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [55]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)

Epoch 1/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1301
Epoch 2/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1174
Epoch 3/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1151
Epoch 4/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1138
Epoch 5/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1127
Epoch 6/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1117
Epoch 7/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1109
Epoch 8/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1102
Epoch 9/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1095
Epoch 10/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1087
Epoch 11/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1075
Epoch 12/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1065
Epoch 13/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1058
Epoch 14/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1051
Epoch 15/30
1455/1455 ━━━━━━━

In [57]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)

364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 0.1060


0.10547296702861786

In [85]:
new_user_id = 5510
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 5
new_childrens = 5
new_comedy = 1
new_crime = 0.5
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 0.5
new_scifi = 1
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [87]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec, len(item_vecs))

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs,  item_vecs, model, u_s, i_s, 
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount = 10)

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


y_p,movie id,rating ave,title,genres
3.91992,55721,4.3,Elite Squad (Tropa de Elite) (2007),Action|Crime|Drama|Thriller
3.91992,96829,4.3,"Hunt, The (Jagten) (2012)",Drama
3.9199,8014,4.25,"Spring, Summer, Fall, Winter... and Spring (Bom yeoreum gaeul gyeoul geurigo bom) (2003)",Drama
3.9199,48516,4.25234,"Departed, The (2006)",Crime|Drama|Thriller
3.91989,116897,4.25,Wild Tales (2014),Comedy|Drama|Thriller
3.91988,80906,4.29167,Inside Job (2010),Documentary
3.91987,71899,4.2,Mary and Max (2009),Animation|Comedy|Drama
3.91986,55442,4.18182,Persepolis (2007),Animation|Drama
3.91985,7361,4.16031,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi
3.91985,6016,4.14667,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
